# Basic stock analysis

# 1 Stock Scraping using yfinance 

In [ ]:
!pip install yfinance

In [ ]:
%matplotlib inline 
import os
import random
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from ipywidgets import interact, fixed, IntSlider
import matplotlib.pyplot as plt
from numpy.linalg import cholesky
import seaborn as sns

We basically plug in the ticker as parameters and we can get the whole history of the particular stock

We used cached_data to avoid repeatedly downloading the stock information again. All stock info would be stored in cached_data in a dictionary format, so that we could save it/reuse them later.

In [ ]:
cached_data={}
def stock_hist(symbol, start=None, end=None, cached_data=cached_data):
    '''Convenience function to get cached data '''
    if not symbol in cached_data:
        cached_data[symbol] = yf.download(symbol)
        print(F'Loaded {symbol} num values = {len(cached_data[symbol])}')
    return cached_data[symbol]

In [ ]:
tickers = ['ITC.NS', 'TCS.NS','HCLTECH.NS','INFY.NS','LT.NS','HDFCBANK.NS']
N = len(tickers)
historical  = pd.concat((stock_hist(symbol)['Adj Close'] for symbol in tickers), axis=1, keys=tickers).dropna(axis=0)

# 2 Portfolio return Backtesting using Interactive Chart

In [ ]:
_ = (historical).plot(figsize=(20,10),title='Stocks Price over time')
historical.head(2)

It’s probably a good practice to backfill any missing values inbetween the dates, and chop off data so that selected stocks have the same length

In [ ]:
returns = (historical/historical.iloc[0]).fillna(method='backfill')
daily_pct_change = np.log(returns.pct_change() + 1)
vols = daily_pct_change.std() * np.sqrt(252)

Backfill is usually used in time-series as it doesn’t spoil the data from the future to the past.

In [ ]:
_ = returns.plot(ylim=(0,130), figsize=(20,8),title='Stocks Cumulative Return over time')

It seems fine with me with no obvious breaks and gaps between data. Also, all stock data is available at t= 0 which is good

In [ ]:
returns['PORTFOLIO'] = returns.iloc[:,0:N].sum(axis=1) /N

For simplicity, we used 1/N equal-weighted portfolio, meaning that each stock has 1/N weight. Feel free to build your own portfolio by replacing returns[‘PORTFOLIO’].

In [ ]:
plt.figure(figsize=(20,10))
lines = plt.plot(returns.drop('PORTFOLIO', axis=1), alpha=0.1)
plt.ylim((0,130))
plt.title('Portfolio Cumulative Return Over Time')
plt.plot(returns['PORTFOLIO'], alpha=1)
plt.legend(returns.columns)
plt.show()

In [ ]:
daily_pct_change = np.log(returns.pct_change() + 1)
vols = daily_pct_change.std() * np.sqrt(252) # Annual Volatility 

tickers=['ITC.NS', 'TCS.NS','HCLTECH.NS','INFY.NS','LT.NS','HDFCBANK.NS']
prices = [stock_hist(symbol)['Adj Close'] for symbol in tickers]
prices = pd.concat((stock_hist(symbol)['Adj Close'] for symbol in tickers), axis=1, keys=tickers).dropna(axis=0)
prices.head(2)

In [ ]:
def diversicheck(symbols, start_day=0, time_horizon=365, **active):
    filtered = [symbol for symbol in symbols if active.get(symbol, True)]
    prices = pd.concat((stock_hist(symbol)['Adj Close'] for symbol in filtered), axis=1, keys=filtered).dropna(axis=0)
    
    
    start_dates = prices.index[0] + datetime.timedelta(days = start_day)
    end_dates = start_dates + datetime.timedelta(days=time_horizon)
    
    prices = prices.loc[start_dates:end_dates]
    
    unit_pos = prices / prices.iloc[0,:]
    basket = unit_pos.sum(axis=1) / unit_pos.shape[1]
    unit_pos.plot(figsize=(20,10),title='Stocks Cumulative Return over time', alpha=0.3)
    basket.plot(figsize=(20,10),legend = True)
    print(f'backtest from {start_dates} to {end_dates}')

In [ ]:
# cached_data ={}
prf_stocks = ['ITC.NS', 'TCS.NS','HCLTECH.NS','INFY.NS','LT.NS','HDFCBANK.NS']
active = dict(zip(prf_stocks, [True] * len(prf_stocks)))
_ = interact(diversicheck, symbols =fixed(prf_stocks), start_day=IntSlider(min=0,max=252*10),
             time_horizon=IntSlider(min=0,value=365,max=252*5),**active)

# 3 Correlation Plot & Risk-Reward Chart

In [ ]:
prf_prices = pd.concat((stock_hist(symbol)['Adj Close'] for symbol in prf_stocks), axis =1, keys=prf_stocks).dropna(axis=0)
prf_returns = (prf_prices.pct_change() + 1)[1:]
log_returns = np.log(prf_returns)
corr = log_returns.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8)) 
sns.heatmap(corr, annot=True)

In [ ]:
vols = log_returns.std() * np.sqrt(252)
avg_return = (prf_returns-1).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(vols, avg_return*252)
ax.set_xlabel('Annualized Volatility')
ax.set_ylabel('Annualized Average Return')
ax.set_xlim(0,0.6)
for i, txt in enumerate(prf_stocks):
    ax.annotate(txt, xy=(vols[i], avg_return[i]*252))

# 4 Monti-Carlo Simulation of Portfolio Return

We basically find that we need to consider more stocks (increasing n to lower volatility) and select stocks from different industry (lower non-systematic risk) to avoid such a highly correlated composition of stocks.

However, there’s nothing really concrete. Hence, we need simulation to see how our portfolio actually performed in numerical terms if history repeats in similar favour (similar return distribution).

In [ ]:
number_of_simulations = 1000
investment_horizon = 60
simulated = pd.DataFrame([(prf_returns.iloc[random.choices(
    range(len(prf_returns)), k=investment_horizon)]).mean(axis=1).cumprod().values 
                         for x in range(number_of_simulations)]).T
simulated.head(3)

Basically, we randomly select the return from past and form synthetic returns for all the stocks over time-horizon (k = 60 days in this case), and we calculated our portfolio based on those synthetic returns. In this example, we simulate for 1,000 times, and get the confidence interval of our portfolio value after 60 days.

In [ ]:
simulated.plot(legend=False, linewidth=1, alpha=0.1, color='blue', figsize=(20,5))
simulated.quantile([0.05, 0.50, 0.95], axis=1).T.plot(figsize=(20,5))

In [ ]:
simulated.to_csv('testing1.csv', index=0) # Save Result

If you like this kernal, Upvote!! Upvote!! Upvote!!

Thank You!